## Чат-бот на модели FastText

In [34]:
!pip install pymorphy2 stop_words annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook

In [4]:
DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/GB_NLP/Chat_Bot/Data/'

### Подготовка данных

In [15]:
import mmap

def get_num_lines(file_path):
    fp = open(file_path, "r+")
    buf = mmap.mmap(fp.fileno(), 0)
    lines = 0
    while buf.readline():
        lines += 1
    return lines

In [16]:
assert True

#Small preprocess of the answers

question = None
written = False

with open(DATA_PATH + "prepared_answers.txt", "w") as fout:
    with open(DATA_PATH + "Otvety.txt", "r") as fin:
        for line in tqdm_notebook(fin, total=get_num_lines(DATA_PATH + "Otvety.txt")):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

<ipython-input-16-c12dbf555266>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin, total=get_num_lines(DATA_PATH + "Otvety.txt")):


  0%|          | 0/7550926 [00:00<?, ?it/s]

In [14]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [18]:
assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0
max_rows = 500000

with open(DATA_PATH + "Otvety.txt", "r") as fin:
    for line in tqdm_notebook(fin, total=max_rows):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > max_rows:
            break

<ipython-input-18-d4100d970fa5>:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin, total=max_rows):


  0%|          | 0/500000 [00:00<?, ?it/s]

Сохраним на случай дальнейшего перезапуска

In [19]:
import pickle
with open(DATA_PATH + 'prepared_sents.pkl', 'wb') as f:
   pickle.dump(sentences, f)

In [5]:
with open(DATA_PATH + 'prepared_sents.pkl', 'rb') as f:
  sentences = pickle.load(f)

Возьмем для обучения предложения где больше 2х слов

In [22]:
sentences = [i for i in sentences if len(i) > 2]

FastText

In [24]:
modelFT = FastText(sentences=sentences, vector_size=300, min_count=1, window=5, workers=8)

In [7]:
MODEL_PATH = '/content/drive/MyDrive/Colab Notebooks/GB_NLP/Chat_Bot/Models/'

In [26]:
modelFT.save(MODEL_PATH + "ft_model")

In [30]:
ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

with open(DATA_PATH + "prepared_answers.txt", "r") as f:
    for line in tqdm_notebook(f, total=max_rows):
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        
        vector_ft = np.zeros(300)
        for word in question:
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index.add_item(counter, vector_ft)
            
        counter += 1
        
        if counter > max_rows:
            break

ft_index.build(10)

<ipython-input-30-518d84432818>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(f, total=max_rows):


  0%|          | 0/500000 [00:00<?, ?it/s]

True

In [31]:
ft_index.save(MODEL_PATH + 'speaker.ann')

True

In [33]:
with open(MODEL_PATH + 'index_map.pkl', 'wb') as f:
   pickle.dump(index_map, f)

In [8]:
with open(MODEL_PATH + 'index_map.pkl', 'rb') as f:
  index_map = pickle.load(f)

In [9]:
modelFT = FastText.load(MODEL_PATH + "ft_model")

In [10]:
ft_index = annoy.AnnoyIndex(300, 'angular')
ft_index.load(MODEL_PATH + 'speaker.ann') 

True

In [11]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 3, )
    return [index_map[i] for i in answers]

Проверим работу модели

In [19]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [32]:
TEXT = "Гриб мухомор"
get_response(TEXT, ft_index, modelFT, index_map)

['грибы -руль! ))). \n',
 'Из дубовиков это, а не польский гриб.<br>См. тут, какой вид более подходит<br>http://grib-info.ru/syedobnie/dubovik.html<br>http://wikigrib.ru/grib/dubovik/. \n',
 'поганка. \n']